In [1]:
!pip install --quiet langchain langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [2]:
import os

os.environ["OPENAI_API_TYPE"] = "azure_ad"
os.environ["OPENAI_API_VERSION"] = "2024-05-01-preview"
# os.environ["AZURE_OPENAI_ENDPOINT"] = "This form the Azure portal - model deployment"
# os.environ["AZURE_OPENAI_API_KEY"] = 'This form the Azure portal - model deployment'
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://mobility-sage-openai-services.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = '9bbf92a17be24c66881eaa5a1c6aa77d'

In [121]:
from langchain_openai import AzureChatOpenAI
from langchain.chains.router import MultiPromptChain
from langchain.chains import LLMChain
from langchain.chains import ConversationChain
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [122]:
llm = AzureChatOpenAI(
            temperature=1,
            azure_deployment="gpt-4o",
        )

# Router Chain

A good way to imagine this is if you have multiple subchains, each of which is specialized for a particular type of input, you could have a router chain, which first decides which subchain to pass it to and then passes it to that chain.

In simple terms if we want out prompt to wear multiple hats or assume differnt roles based on the user input, we can use the router chain.

In [123]:
# Create multiple prompts

physics_template = """
  You are a very smart physics professor. \
  You are great at answering questions about physics in a concise\
  and easy to understand manner. \
  When you don't know the answer to a question you admit\
  that you don't know.

  Here is a question:
  {input}
"""


math_template = """
  You are a very good mathematician. \
  You are great at answering math questions. \
  You are so good because you are able to break down \
  hard problems into their component parts,
  answer the component parts, and then put them together\
  to answer the broader question.

  Here is a question:
  {input}
"""

history_template = """
  You are a very good historian. \
  You have an excellent knowledge of and understanding of people,\
  events and contexts from a range of historical periods. \
  You have the ability to think, reflect, debate, discuss and \
  evaluate the past. You have a respect for historical evidence\
  and the ability to make use of it to support your explanations \
  and judgements.

  Here is a question:
  {input}
"""


computerscience_template = """
  You are a successful computer scientist.\
  You have a passion for creativity, collaboration,\
  forward-thinking, confidence, strong problem-solving capabilities,\
  understanding of theories and algorithms, and excellent communication \
  skills. You are great at answering coding questions. \
  You are so good because you know how to solve a problem by \
  describing the solution in imperative steps \
  that a machine can easily interpret and you know how to \
  choose a solution that has a good balance between \
  time complexity and space complexity.

  Here is a question:
  {input}
"""

In [124]:
# Build a prompt descriptor

prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

## Build the Destination chains.

## These are the chains that will be called by the router chain.

In [127]:
destination_chain = {}

for p_info in prompt_infos:
  name = p_info['name']
  prompt_template = p_info['prompt_template']

  prompt = ChatPromptTemplate.from_template(
      template=prompt_template
  )

  chain = LLMChain(
      llm = llm,
      prompt = prompt
  )

  destination_chain[name] = chain



In [128]:
destination_chain.keys()

dict_keys(['physics', 'math', 'History', 'computer science'])

In [130]:
default_prompt = ChatPromptTemplate.from_template(
    "{input}"
    )

In [131]:
default_chain = LLMChain(
    llm = llm,
    prompt = default_prompt,
    output_key = "text"
)

In [132]:
default_prompt = PromptTemplate.from_template("{blackhole}")
default_prompt

PromptTemplate(input_variables=['blackhole'], template='{blackhole}')

In [133]:
destinations = [f"{p['name']} : {p['description']}" for p in prompt_infos]
destinations

['physics : Good for answering questions about physics',
 'math : Good for answering math questions',
 'History : Good for answering history questions',
 'computer science : Good for answering computer science questions']

In [134]:
destinations_str = '\n'.join(destinations)
destinations_str

'physics : Good for answering questions about physics\nmath : Good for answering math questions\nHistory : Good for answering history questions\ncomputer science : Good for answering computer science questions'

# Router Chain.

In [98]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
# from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

### Defining Custom Multi Prompt Router Template

In [135]:

MULTI_PROMPT_ROUTER_TEMPLATE = """
  Given a raw text input to a \
  language model select the model prompt best suited for the input. \
  You will be given the names of the available prompts and a \
  description of what the prompt is best suited for. \
  You may also revise the original input if you think that revising\
  it will ultimately lead to a better response from the language model.

  << FORMATTING >>
  Return a markdown code snippet with a JSON object formatted to look like:
  ```json
  {{{{
      "destination": string \ name of the prompt to use or "DEFAULT"
      "next_inputs": string \ a potentially modified version of the original input
  }}}}
  ```

  REMEMBER: "destination" MUST be one of the candidate prompt \
  names specified below OR it can be "DEFAULT" if the input is not\
  well suited for any of the candidate prompts.
  REMEMBER: "next_inputs" can just be the original input \
  if you don't think any modifications are needed.

  << CANDIDATE PROMPTS >>
  {destinations}

  << INPUT >>
  {{input}}

  << OUTPUT (remember to include the ```json)>>
"""


In [136]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations = destinations_str
)

print(router_template) # built in  MULTI_PROMPT_ROUTER_TEMPLATE from langchain


  Given a raw text input to a   language model select the model prompt best suited for the input.   You will be given the names of the available prompts and a   description of what the prompt is best suited for.   You may also revise the original input if you think that revising  it will ultimately lead to a better response from the language model.

  << FORMATTING >>
  Return a markdown code snippet with a JSON object formatted to look like:
  ```json
  {{
      "destination": string \ name of the prompt to use or "DEFAULT"
      "next_inputs": string \ a potentially modified version of the original input
  }}
  ```

  REMEMBER: "destination" MUST be one of the candidate prompt   names specified below OR it can be "DEFAULT" if the input is not  well suited for any of the candidate prompts.
  REMEMBER: "next_inputs" can just be the original input   if you don't think any modifications are needed.

  << CANDIDATE PROMPTS >>
  physics : Good for answering questions about physics
math : 

In [137]:
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=['input'],
    output_parser=RouterOutputParser()
)

In [138]:
router_chain = LLMRouterChain.from_llm(
    llm=llm,
    prompt=router_prompt
)

In [139]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chain,
    default_chain=default_chain,
    verbose=True
)

In [140]:
chain.run('Explain Generative AI')



> Entering new MultiPromptChain chain...
computer science: {'input': 'Explain Generative AI'}
> Finished chain.


"Generative AI refers to a class of artificial intelligence systems designed to create new, original content. Unlike traditional AI, which performs tasks such as classification or prediction based on existing data, generative AI aims to generate new examples from learned patterns. This can include text, images, music, and more. Let's delve into the specifics.\n\n### Key Components\n\n1. **Models and Architectures**:\n    - **Generative Adversarial Networks (GANs)**: Comprises two neural networks, a generator, and a discriminator, which work against each other. The generator creates content, and the discriminator evaluates it, pushing the generator to create more realistic content over time.\n    - **Variational Autoencoders (VAEs)**: Involves an encoder that compresses input data into a latent space and a decoder that reconstructs data from the latent space. Useful for generating new, similar data points.\n    - **Transformers**: Especially popular in natural language processing, trans

In [141]:
chain.run('What is a black hole !!')



> Entering new MultiPromptChain chain...
physics: {'input': 'What is a black hole?'}
> Finished chain.


"A black hole is a region of space where gravity is so strong that nothing, not even light, can escape from it. This extreme gravitational pull is a result of a large amount of mass being concentrated into a very small area. \n\nThe boundary surrounding a black hole is called the event horizon. Once something crosses this boundary, it cannot escape the black hole's gravitational pull. The core of a black hole is known as the singularity, where density becomes infinite and the laws of physics as we currently understand them break down.\n\nBlack holes can form from the remnants of massive stars that have undergone supernova explosions, and there are also supermassive black holes at the centers of most galaxies, including our own Milky Way.\n\nIf you have any more specific questions about black holes, feel free to ask!"